In [16]:
using Turing
using LimberJack
using LinearAlgebra
using CSV
using NPZ
using Plots
using FITSIO
using ForwardDiff
using BenchmarkTools
using DelimitedFiles

In [17]:
files = npzread("../data/DESY1_cls/gcgc_gcwl_wlwl.npz")
#Cls_meta = cls_meta(files)

Dict{String, Array} with 42 entries:
  "ls_1022"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_1321"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_1313"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_2123"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_2223"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_1021"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_1020"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_2222"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "inv_cov"   => [5.40328e8 9.23364e8 … 7.45929e11 9.18557e11; 9.23364e8 1.1092…
  "ls_2122"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_1221"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 255.5, 2…
  "ls_2323"   => [14.5, 44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5, 2

In [18]:
cov_tot = files["cov"]
inv_cov_tot = files["inv_cov"]
data_vector = files["cls"];

In [19]:
#                Omega_b,           Omega_c          n_s,                 h,
#                DESgc__0_gc_b,     DESgc__1_gc_b,   DESgc__2_gc_b,    DESgc__3_gc_b,    DESgc__4_gc_b,
#                DESgc__0_dz,      DESgc__1_dz,      DESgc__2_dz,      DESgc__3_dz,      DESgc__4_dz,
#                wl_ia_A,          wl_ia_eta, 
#                DESwl__0_wl_m,    DESwl__1_wl_m,    DESwl__2_wl_m,    DESwl__3_wl_m,
#                DESwl__0_dz,      DESwl__1_dz,      DESwl__2_dz,      DESwl__3_dz,
#                sigma_8,          Omega_m
carlos_params1 = [4.426868e-02,     2.093138e-01,     8.963611e-01,     8.495440e-01,
                 1.343888e+00,    1.639047e+00,      1.597174e+00,     1.944583e+00,     2.007245e+00,
                -4.679383e-03,   -2.839996e-03,      1.771571e-03,     1.197051e-03,    -5.199799e-03,
                 2.389208e-01,   -6.435288e-01, 
                 1.802722e-03,   -5.508994e-03,     1.952514e-02,    -1.117726e-03,
                -1.744083e-02,    6.777779e-03,    -1.097939e-03,    -4.912315e-03,
                 8.536883e-01,    2.535825e-01];

In [20]:
cls_carlos = npzread("/home/jaime/PhD/LimberJack.jl/chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info.npz");
cls_carlos2 = npzread("/home/jaime/PhD/LimberJack.jl/chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info_nointerp.npz");
cls_carlos3 = npzread("/home/jaime/PhD/LimberJack.jl/chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info_nointerp_EH.npz");
cls_carlos4 = npzread("/home/jaime/PhD/LimberJack.jl/chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info_nointerp_EH_noconv.npz");

In [21]:
pars = carlos_params1
nuisances = Dict("b0" => pars[5],
                 "b1" => pars[6],
                 "b2" => pars[7],
                 "b3" => pars[8],
                 "b4" => pars[9],
                 "dz_g0" => pars[10],
                 "dz_g1" => pars[11],
                 "dz_g2" => pars[12],
                 "dz_g3" => pars[13],
                 "dz_g4" => pars[14],
                 "dz_k0" => pars[21],
                 "dz_k1" => pars[22],
                 "dz_k2" => pars[23],
                 "dz_k3" => pars[24],
                 "m0" => pars[17],
                 "m1" => pars[18],
                 "m2" => pars[19],
                 "m3" => pars[20],
                 "A_IA" => pars[15],
                 "alpha_IA" => pars[16]);

In [22]:
function make_cls(pars)
    nuisances = Dict("b0" => pars[5],
                     "b1" => pars[6],
                     "b2" => pars[7],
                     "b3" => pars[8],
                     "b4" => pars[9],
                     "dz_g0" => pars[10],
                     "dz_g1" => pars[11],
                     "dz_g2" => pars[12],
                     "dz_g3" => pars[13],
                     "dz_g4" => pars[14],
                     "dz_k0" => pars[21],
                     "dz_k1" => pars[22],
                     "dz_k2" => pars[23],
                     "dz_k3" => pars[24],
                     "m0" => pars[17],
                     "m1" => pars[18],
                     "m2" => pars[19],
                     "m3" => pars[20],
                     "A_IA" => pars[15],
                     "alpha_IA" => pars[16])

    Ωm = pars[26]
    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]
    s8 = pars[25]
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="EisHu", Pk_mode="Halofit")#, 
                                     #nk=500, nz=500, nz_pk=500)

    return Theory(cosmology, Cls_meta, files; Nuisances=nuisances).cls

end

make_cls (generic function with 1 method)

In [53]:
cosmology = LimberJack.Cosmology(0.3, Wb, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
Theory(cosmology, files).cls

LoadError: UndefVarError: cosmology not defined

In [50]:
function diff_cls(Wb)
    cosmology = LimberJack.Cosmology(0.3, Wb, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls = Theory(cosmology, files).cls
    return cls
end

function numd_cls(Wb)
    cosmology1 = LimberJack.Cosmology(0.3, Wb, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cosmology2 = LimberJack.Cosmology(0.3, Wb-0.0001, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls1 = Theory(cosmology1, files).cls
    cls2 = Theory(cosmology2, files).cls
    return (cls1 .- cls2) ./ 0.0001
end


function diff_cls_nuisance(dz)
    nuisances = Dict(#"b0" => 2.0,
                     "dz_g2"=> dz)
    cosmology = LimberJack.Cosmology(0.3, 0.05, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls = Theory(cosmology, files; Nuisances=nuisances).cls
    return cls
end

function numd_cls_nuisance(dz)
    nuisances1 = Dict(#"b0" => 2.0,
                      "dz_g2"=> dz)
    nuisances2 = Dict(#"b0" => 2.0,
                      "dz_g2"=> dz.-0.0005)
    cosmology = LimberJack.Cosmology(0.3, 0.05, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls1 = Theory(cosmology, files; Nuisances=nuisances1).cls
    cls2 = Theory(cosmology, files; Nuisances=nuisances2).cls
    return (cls1 .- cls2) ./ 0.0005
end



numd_cls_nuisance (generic function with 1 method)

In [51]:
mean(abs.(numd_cls(0.05)./ForwardDiff.derivative(diff_cls, 0.05) .-1))

LoadError: ArgumentError: broadcasting over dictionaries and `NamedTuple`s is reserved

In [49]:
comp = abs.(numd_cls(0.05)./ForwardDiff.derivative(diff_cls, 0.05) .-1)
plot(comp)

LoadError: ArgumentError: broadcasting over dictionaries and `NamedTuple`s is reserved

In [26]:
nume = numd_cls_nuisance(-0.01)
anal = ForwardDiff.derivative(diff_cls_nuisance, -0.01)
n = length(nume)
comp = zeros(Float64, n)

for i in 1:n 
    if (nume[i] != 0)
        comp[i] = @.(abs(anal[i]/nume[i] - 1))
    end
end

diff = []
for i in 1:n 
    if (nume[i] != 0)
        push!(diff, @.(abs(nume[i]-anal[i])))
    end
end
println(mean(diff))

plot(comp)

LoadError: UndefVarError: Cls_meta not defined

In [27]:
cls_jaime1 = make_cls(carlos_params1);

LoadError: UndefVarError: Cls_meta not defined

In [28]:
open("jaime_cls.txt", "w") do io
       writedlm(io, cls_jaime1)
end

LoadError: UndefVarError: cls_jaime1 not defined

In [29]:
compa1 = @. abs(cls_carlos["cls"]-cls_jaime1)/cls_carlos["cls"];
compa2 = @. abs(cls_carlos2["cls"]-cls_jaime1)/cls_carlos2["cls"];
compa3 = @. abs(cls_carlos3["cls"]-cls_jaime1)/cls_carlos3["cls"];
compa4 = @. abs(cls_carlos4["cls"]-cls_jaime1)/cls_carlos4["cls"];

println("Mean discrepancy default   ", mean(compa1))
println("Mean discrepancy no itp    ", mean(compa2))
println("Mean discrepancy no itp+EH ", mean(compa3))
println("Mean discrepancy no itp+EH+noconv ", mean(compa4))

LoadError: UndefVarError: cls_jaime1 not defined

In [30]:
chi2_carlos = cls_carlos["chi2_nolp"]
chi2_carlos2 = cls_carlos2["chi2_nolp"]
chi2_carlos3 = cls_carlos3["chi2_nolp"]
chi2_carlos4 = cls_carlos4["chi2_nolp"]
println(chi2_carlos)
println(chi2_carlos2)
println(chi2_carlos3)
println(chi2_carlos4)

495.62231927779936
495.00947244084654
495.89420547809317
497.2051355000593


In [31]:
chi2_jaime1 = dot((cls_jaime1 .- data_vector), inv_cov_tot  * (cls_jaime1 .- data_vector))
println(chi2_jaime1)

LoadError: UndefVarError: cls_jaime1 not defined

In [32]:
(chi2_jaime1-chi2_carlos4)/chi2_carlos4

LoadError: UndefVarError: chi2_jaime1 not defined

In [33]:
plot(compa4, label="No itp + EH + no Conv", yaxis=:log)

LoadError: UndefVarError: compa4 not defined

In [42]:
@model function model(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 = 0.81 #~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)
    
    b0 = 2.0
    dz_g0 ~ TruncatedNormal(0.008, 0.007, -2.0, 2.0)
    dz_g1 ~ TruncatedNormal(-0.005, 0.007, -2.0, 2.0)
    dz_g2 ~ TruncatedNormal(0.006, 0.006, -2.0, 2.0)
    dz_g3 ~ TruncatedNormal(0.0, 0.010, -2.0, 2.0)
    dz_g4 ~ TruncatedNormal(0.0, 0.010, -2.0, 2.0)

    nuisances = Dict(#"b0" => b0,
                     "dz_g0" => dz_g0,
                     "dz_g1" => dz_g1,
                     "dz_g2" => dz_g2,
                     "dz_g3" => dz_g3,
                     "dz_g4" => dz_g4)
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, s8, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    theory = Theory(cosmology, files;
                    nz_path="../data/DESY1_cls/nzs_fiducial",
                    Nuisances=nuisances)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

In [43]:
iterations = 300
nadapts = 100;

In [24]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, true, false, true)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, true, false, true)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, fa

Sampling:  10%|███▉                                     |  ETA: 0:05:37┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:37


LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
     [1] [0m[1mangularCℓs[22m[0m[1m([22m[90mcosmo[39m::[0mCosmology, [90mt1[39m::[0mNumberCountsTracer, [90mt2[39m::[0mWeakLensingTracer, [90mℓs[39m::[0mVector[90m{Float64}[39m[0m[1m)[22m
    [90m   @ [39m[35mLimberJack[39m [90m~/PhD/LimberJack.jl/src/[39m[90m[4mspectra.jl:30[24m[39m
     [2] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m~/PhD/LimberJack.jl/src/[39m[90m[4mtheory.jl:72[24m[39m[90m [inlined][39m
     [3] [0m[1m(::LimberJack.var"#25#threadsfor_fun#23"{Cosmology, cls_meta{Vector{Vector{Int64}}}, Dict{String, Array}, Vector{ForwardDiff.Dual{ForwardDiff.Tag{Turing.Core.var"#f#7"{DynamicPPL.TypedVarInfo{NamedTuple{(:Ωm, :dz_g0, :dz_g1, :dz_g2, :dz_g3, :dz_g4), Tuple{DynamicPPL.Metadata{Dict{AbstractPPL.VarName{:Ωm, Tuple{}}, Int64}, Vector{Uniform{Float64}}, Vector{AbstractPPL.VarName{:Ωm, Tuple{}}}, Vector{Float64}, Vector{Set{

In [44]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

LoadError: MethodError: no method matching Theory(::Cosmology, ::Dict{String, Array}; nz_path="../data/DESY1_cls/nzs_fiducial", Nuisances=Dict("dz_g0" => -0.007424084241504165, "dz_g1" => -0.00829466998186491, "dz_g4" => 0.011798619127634276, "dz_g2" => 0.007292462353226616, "dz_g3" => 0.004882280887009468))
[0mClosest candidates are:
[0m  Theory(::Cosmology, ::Any, [91m::Any[39m; Nuisances) at ~/PhD/LimberJack.jl/src/theory.jl:29[91m got unsupported keyword argument "nz_path"[39m
[0m  Theory(::Any, ::Any) at ~/PhD/LimberJack.jl/src/theory.jl:15[91m got unsupported keyword arguments "nz_path", "Nuisances"[39m